In [43]:
import requests
import json
from bs4 import BeautifulSoup

def get_page(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, "lxml")
    json_script = soup.find("script", type="application/ld+json")

    # If we got the content
    if json_script is not None:
        script_content = json.loads(json_script.text)
        # If the last thing in the navigation is "Cocktail Recipes"
        if ("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name']):
            script_content = json.loads(json_script.text)
            drink_name = script_content[1]['name']
            ingredients = script_content[1]["recipeIngredient"]
            instr = script_content[1]["recipeInstructions"]

            step_list = []
            for step in instr:
                step_list.append(step["text"])

            with open("martha_stewart.json", 'a') as outfile:
                data = {"name": drink_name, "ingredients": ingredients, "instructions": step_list}


In [46]:
get_page("https://www.marthastewart.com/1165530/french-martini")

French Martini
['1 1/2 ounces vodka', '1/2 ounce Chambord', '3/4 ounces pineapple juice', 'Lemon twist, for garnish']
Fill a cocktail shaker halfway with ice. Add in vodka, Chambord, and pineapple juice. Shake well. Strain into a 4-ounce martini glass and garnish with lemon twist.


In [42]:
req = requests.get("https://www.marthastewart.com/314342/extra-dirty-martini")
soup = BeautifulSoup(req.text, "lxml")
json_script = soup.find("script", type="application/ld+json")
script_content = json.loads(json_script.text)
print("Cocktail Recipes" == script_content[0]['itemListElement'][-1]['item']['name'])

True


In [ ]:
# Esquire has somewhat semantic markup for recipes
# Ingredients are in a ingredients-body
# Instructions are in a directions-body
# Can probably filter URLs by having "drinks" in them, like
# <link href="https://www.esquire.com/food-drink/drinks/recipes/a3683/negroni-drink-recipe/" rel="canonical">

    

In [ ]:
# https://www.thecocktaildb.com/api.php
# Probably has some sort of rate limiting or something.
# May end up being an excuse to write a proxy spinner


In [ ]:
#https://makemeacocktail.com/recipes/Gin-cocktails/?ing=61
#https://kindredcocktails.com/cocktail/bananarac
#https://www.thecocktailproject.com/search-recipes
# URLS of the form https://www.diffordsguide.com/cocktails/recipe
#https://www.cocktailcontessa.com
#https://www.allrecipes.com/recipes/133/drinks/cocktails/
#https://www.seriouseats.com/drink-recipes-5117861
#https://imbibemagazine.com/category/recipes/cocktails-spirits-recipes/
#https://mrbostondrinks.com/recipes/blue-lagoon

In [52]:
url = "https://www.pinterest.com/pin/create/link/?url=https://www.marthastewart.com/2126005/halloween-cake-recipes?slide=094fbdda-1f06-42ae-8722-fe180a2413fd%3Futm_source=pinterest.com%26utm_medium=social%26utm_campaign=social-share-gallery%26utm_content=20210921&media=https%3A%2F%2Fimagesvc.meredithcorp.io%2Fv3%2Fmm%2Fimage%3Furl%3Dhttps%253A%252F%252Fassets.marthastewart.com%252Fstyles%252Fwmax-750%252Fd33%252Fcrawly-cake-phobias-1011mld107647%252Fcrawly-cake-phobias-1011mld107647_vert_0_horiz.jpg%253Fitok%253Df2RnN138&description=Creepy-Crawly%20Cake%20"
import re
matcher = re.compile("(http|https)://www\.marthastewart\.com")
print(re.match(matcher, url))
url = "https://www.marthastewart.com/2138473/pastry-chef-dominique-ansel-shares-lessons-learned-martha-stewart"
print(re.match(matcher, url))


None
<_sre.SRE_Match object; span=(0, 29), match='https://www.marthastewart.com'>


In [2]:
import requests
from bs4 import BeautifulSoup
req = requests.get("https://mrbostondrinks.com/recipes/blue-lagoon-2000")
soup = BeautifulSoup(req.text, "lxml")


In [3]:
req.text

'<!doctype html>\n<html class="no-js" ng-app="mrBostonRecipes" lang="en" ng-strict-di>\n  <head>\n    <!-- Google Tag Manager -->\n    <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n    new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n    j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n    \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n    })(window,document,\'script\',\'dataLayer\',\'GTM-NSKCGD4\');</script>\n    <!-- End Google Tag Manager -->\n    <meta charset="utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\n    <title ng-bind-template="{{viewTitle}} | Mr. Boston Drinks"></title>\n    <base href="/index.html">\n    <meta property="og:site_name" content="Mr. Boston Drinks"/>\n    <meta name="fragment" content="!">\n    <meta property="fb:app_id" content="948323305227233" />\n    <meta name="p:domain_verify" conten

In [ ]:
from bs4 import BeautifulSoup
with open("./data/index.html", 'r') as infile:
    html = infile.read()
    bs = BeautifulSoup(html)
    print(bs.find("h1", attrs={"class": "recipe-name"}).text)

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random

In [ ]:
# Request to their backend that the "Load more" button does
responses = {}
for ii in range(30):
    req_data = {"context":"recipes","number":12, "value":0,"post_type":"recipe","page":ii}
    responses[ii] = requests.post("https://www.themixer.com/en-us/wp-json/site/v1/load-more", data=req_data)

In [ ]:
# Clean up the results
import json
import re
recipe_url_matcher = re.compile("https://www\.themixer\.com/en-us/recipes/[^/]*/")

recipes = []
for r in responses.values():
    data = json.loads(r.text)
    as_html = data["html"]
    urls = (re.findall(recipe_url_matcher, as_html))
    if len(urls) > 0:
        recipes.extend(urls)

In [ ]:
# Load up some random page
test_url = recipes[20]
page = requests.get(test_url)

In [30]:
bs = BeautifulSoup(page.text)

In [54]:
def mixer_to_dict(bs):
    name = bs.find("h1", attrs={"class", "title"}).text.strip()
    
    ingred_list = []
    ingredients = bs.find_all("div", attrs={"class": "d-flex align-items-top justify-content-between export-ingredient-item"})
    for ing in ingredients:
        ingredient = {}
        quant = ing.find("span", attrs = {"class":"unity_quantity"})
        if quant is not None:
            ingredient["amount"] = quant.text.strip()
        unit = ing.find("span", attrs = {"class":"unity_value"})
        if unit is not None:
            ingredient["unit"] = unit.text.strip()
        content = ing.find("div", attrs={"class": "content"})
        if content is not None:
            ingredient["name"] = content.text.strip()

        ingred_list.append(ingredient)

    instr_list = []
    instructions = bs.find_all("div", attrs={"class", "content fst-italic export-instructions-item"})
    for i in instructions:
        instr_list.append(i.text.replace("\n", ""))

    cocktail = {"name":name, "ingredients":ingred_list, "instructions":instr_list}
    return cocktail

In [55]:
import random # Don't go as hard as possible
import time

mixer_coctails = []
for url in recipes:
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    cocktail = mixer_to_dict(soup)
    mixer_coctails.append(cocktail)
    #time.sleep(random.random() * 2)



In [56]:
with open("./data/themixer_drinks.json", 'w') as outfile:
    json.dump(mixer_coctails, outfile)

In [ ]:
# Now let's go pillage Cocktail-society.com, starting with iterating all the index pages and then all the sub-pages on them
liquors = ["brandy", "gin", "mezcal", "rum", "tequila", "vodka", "whiskey"] 
# "other" and "shots" don't conform to the pattern, get them next
recipes = set()

def get_recipes(url):
    page = requests.get(url)
    bs = BeautifulSoup(page.text)
    links = bs.find_all("a", attrs={"class":"ct-link"})
    for link in links:
        if link['href'].startswith("https://cocktail-society.com/recipes/"):
            recipes.add(link['href'])
    next_page = bs.find("a", attrs={"class": "next page-numbers"})
    if next_page is not None:
        # LOL recursion machine go brrrrrrr
        get_recipes(next_page['href'])

for liquor in liquors:
    get_recipes(f"https://cocktail-society.com/category/recipes/{liquor}-cocktails/")
    


In [64]:
print(recipes)
len(recipes)

{'https://cocktail-society.com/recipes/when-the-smoke-cleared/', 'https://cocktail-society.com/recipes/gin-cocktails/ramos-gin-fizz/', 'https://cocktail-society.com/recipes/chocolate-martini/', 'https://cocktail-society.com/recipes/long-island-iced-tea/', 'https://cocktail-society.com/recipes/mezcal-dante-cocktail/', 'https://cocktail-society.com/recipes/glasgow-mule/', 'https://cocktail-society.com/recipes/manhattan-vs-old-fashioned-cocktail/', 'https://cocktail-society.com/recipes/christmas-mezcal-cocktail/', 'https://cocktail-society.com/recipes/jungle-bird/', 'https://cocktail-society.com/recipes/jack-frost/', 'https://cocktail-society.com/recipes/what-is-grog/', 'https://cocktail-society.com/recipes/bourbon-eggnog/', 'https://cocktail-society.com/recipes/singapore-sling-cocktail/', 'https://cocktail-society.com/recipes/classic-gin-gimlet-cocktail/', 'https://cocktail-society.com/recipes/succulent-blood-cocktail/', 'https://cocktail-society.com/recipes/sea-breeze/', 'https://cockta

246

In [73]:
url = "https://cocktail-society.com/recipes/devils-soul-cocktail/"
page= requests.get(url)

In [90]:
# There's a script tag that is basically a JSON representation of some of the content of the page, and seems to
# include the recipe itself. Going to try pulling the parts I want out of that and dumping that to JSON instead 
# of HTML parsing the soup. 
society_cocktails = []
for url in recipes:
    page = requests.get(url)
    bs = BeautifulSoup(page.text)
    json_tag = bs.find("script", class_="yoast-schema-graph")
    if json_tag is None:
        print(f"No JSON for {url}")
        continue
    data = json.loads(json_tag.text)

    cocktail = {}
    for entry in data["@graph"]:
        if entry["@type"] == "NewsArticle":
            cocktail["name"] = entry["headline"]
        if entry["@type"] == "Recipe":
            cocktail["ingredients"] = entry["recipeIngredient"]
            cocktail["instructions"] = []
            for instr in entry["recipeInstructions"]:
                cocktail["instructions"].append(instr["name"])

    society_cocktails.append(cocktail)
    time.sleep(random.random() * 3)

with open("./data/cocktail_society.json", 'w') as outfile:
    json.dump(society_cocktails, outfile, indent=4)

In [94]:
# Diffords is just firing requests like 
# https://www.diffordsguide.com/api/dev/frontend/box/infinite-scroll?limit=24&target=consumer&lang=gb&sub_category=cocktails&oldest=1729328847&_a=3
# with the oldest incrementing, next value was 1727985924, then 1724266304

# The current unix epoch is 1733804481, so these are way earlier than that, about 6 months to be exact. So what if I do a request for a limit of
# tens of thousands and a unix time of the early '90s... 
#data = requests.get("https://www.diffordsguide.com/api/dev/frontend/box/infinite-scroll?limit=24000&target=consumer&lang=gb&sub_category=cocktails&oldest=755497346&_a=3")
# The answer is that they send a page that says "We've detected suspicious traffic coming from your location."

start_epoch = 1732889443
end_epoch = 1378915509

step = int((start_epoch - end_epoch)/70)
# The A parameter also varies from 1 - 70

gets = {}
for aa in range(70):
    url = f"https://www.diffordsguide.com/api/dev/frontend/box/infinite-scroll?limit=24&target=consumer&lang=gb&sub_category=cocktails&oldest={start_epoch - (aa * step)}&_a={aa+1}"
    gets[aa] = requests.get(url)
    time.sleep(random.random()*2)

In [ ]:
print(gets[10].text)
# I ended up just logging accessess to this page with Firefox dev tools. 

<!DOCTYPE html>
<html lang="en">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	<meta name="viewport" content="width=device-width">
	<title>Is everything okay?</title>
	<link rel="stylesheet" href="/assets/fonts/publico/config.css">
	<link href="/assets/dist/default.min.css" media="screen" rel="stylesheet" type="text/css" >
	<link rel="shortcut icon" type="image/x-icon" href="/favicon.ico" />
	<link rel="apple-touch-icon" href="/touch-icon-iphone.png">
	<link rel="apple-touch-icon" sizes="76x76" href="/touch-icon-ipad.png">
	<link rel="apple-touch-icon" sizes="120x120" href="/touch-icon-iphone-retina.png">
	<link rel="apple-touch-icon" sizes="152x152" href="/touch-icon-ipad-retina.png">
</head>
<body class="fill relative bg-box" style="background-image: url('/assets/images/themes/default_v5/global/diagonal_texture.png')">
	<div class="grid-container flex-container align-center-middle">
		<div class="border bg-white" style="max-width:40rem">
			<div class="

In [2]:
# Mybartender.com has a main page that has links on it to all the spirits, and then the spirit pages are of the form
# https://mybartender.com/drinks/with/amaretto/page/2/

# First, get all the categories
url = "https://mybartender.com/cocktail-recipes/"
page = requests.get(url)
bs = BeautifulSoup(page.text)
categories = [a['href'] for a in bs.find_all(href=re.compile(".*drinks/with/.*"))]

all_recipes = []

for cat_link in categories:
    print(f"Have {len(all_recipes)} urls going into {cat_link}")
    time.sleep(random.random() * 3)
    # Handle the first page
    cat_page = requests.get(cat_link)
    cat_soup = BeautifulSoup(cat_page.text)
    recipes = [a['href'] for a in cat_soup.find_all(href=re.compile(".*/recipe/.*"))]
    all_recipes.extend(recipes)
    # Handle subsequent pagination up to 100 pages or the end, whichever comes first
    suffix_index = 2
    while suffix_index < 100:
        time.sleep(random.random() * 3)
        cat_page = requests.get(cat_link + f"page/{suffix_index}")
        cat_soup = BeautifulSoup(cat_page.text)
        recipes = [a['href'] for a in cat_soup.find_all(href=re.compile(".*/recipe/.*"))]
        all_recipes.extend(recipes)
        if len(recipes) < 36: 
            # Pages of the form "https://mybartender.com/drinks/with/gin/page/8" get 36 
            # recipes. If it's less than 36, it's the last page, so we have a heuristic 
            # for detecting the end of the pagination
            print(f"Last page out at {suffix_index}")
            break
        else:
            suffix_index += 1




Have 0 urls going into https://mybartender.com/drinks/with/absinthe/
Last page out at 2
Have 48 urls going into https://mybartender.com/drinks/with/amaretto/
Last page out at 3
Have 135 urls going into https://mybartender.com/drinks/with/amaro/
Last page out at 2
Have 153 urls going into https://mybartender.com/drinks/with/aperol/
Last page out at 3
Have 258 urls going into https://mybartender.com/drinks/with/baileys/
Last page out at 2
Have 291 urls going into https://mybartender.com/drinks/with/blue-curacao/
Last page out at 8
Have 582 urls going into https://mybartender.com/drinks/with/bourbon/
Last page out at 11
Have 981 urls going into https://mybartender.com/drinks/with/brandy/
Last page out at 5
Have 1143 urls going into https://mybartender.com/drinks/with/campari/
Last page out at 3
Have 1242 urls going into https://mybartender.com/drinks/with/chambord/
Last page out at 2
Have 1281 urls going into https://mybartender.com/drinks/with/champagne/
Last page out at 2
Have 1356 urls

In [4]:
import json
with open("./data/mybartender_urls.json", 'w') as outfile:
    json.dump(sorted(list(set(all_recipes))), outfile, indent=4)

In [5]:
# Cleaning up the Diffords URLs from the data log
with open("./data/www.diffordsguide.com_Archive [24-12-09 23-47-13].har", 'r') as infile: 
    log = json.load(infile)

In [ ]:
# Get all the URLS out of the diffortds log and dump them to a file
urls = []
for entry in log['log']['entries']:
    text = json.loads(entry['response']['content']['text'])
    for box in text['boxes']:
        urls.append(box['href'])

with open("./data/diffords_urls.json", 'w') as outfile:
    json.dump(sorted(list(set(urls))), outfile, indent=4)


In [30]:
headers = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0",
           "Accept": "*/*",
           "Accept-Language": "en-US,en;q=0.5",
           "Accept-Encoding": "gzip, deflate, br, zstd",
           "X-Requested-With": "XMLHttpRequest",
           "Connection": "keep-alive",
           "Sec-Fetch-Dest": "empty",
           "Sec-Fetch-Mode": "cors",
            "Sec-Fetch-Site": "same-origin",
            "Priority": "u=0",
            "TE": "trailers",
            "Referer":"https://eat.my.shorts.com/random/nonsense"
            }

In [31]:
# Get a page out of Difford's book
url = "https://www.diffordsguide.com/cocktails/recipe/9199/aged-grappa-espresso-martini"
page = requests.get(url)
bs = BeautifulSoup(page.text)

In [42]:
def parse_diffords(html):
    # Getting a recipe from a Diffords page
    bs = BeautifulSoup(html)
    json_tag = bs.find("script", type="application/ld+json")
    if json_tag is None:
        print(f"No JSON for {url}")

    data = json.loads(json_tag.text)

    cocktail = {}
    cocktail['name'] = data['name']
    cocktail["ingredients"] = data["recipeIngredient"]
    cocktail["instructions"] = []
    for instr in data["recipeInstructions"]:
        cocktail["instructions"].append(instr["text"])
    return(cocktail)


In [45]:
# If you won't give up your secrets for requests, then you must face selenium.
# Bot protection is a fuck, all your cocktails are belong to me. 
from selenium import webdriver
driver = webdriver.Firefox()

url_list = []
with open("./data/diffords_urls.json", 'r') as infile:
    url_list = json.load(infile)

diff_cocktails = []
for url in url_list:
    url = "https://diffordsguide.com" + url
    driver.get(url)
    driver.implicitly_wait(0.5)
    html = driver.page_source
    diff_cocktails.append((parse_diffords(html)))
    time.sleep(random.random() * 2)

driver.quit()

with open("./data/diffords.json", 'w') as outfile:
    json.dump(diff_cocktails, outfile, indent=4)

In [66]:
# Get HTML from mybartender and parse it. They have a similar JSON structure in the page to 
# Diffords and such, so get it from that instead of dealing directly with HTML. 
recipes = []
        
with open("./data/mybartender_urls.json", 'r') as infile:
    urls = json.load(infile)
    for url in urls:
        time.sleep(random.random() * 2)
        page = requests.get(url)
        bs = BeautifulSoup(page.text)
        tag = bs.find("script", class_="rank-math-schema-pro")
        data = json.loads(tag.text)
        for item in data["@graph"]:
            if item["@type"] == 'Recipe':
                cocktail = {}
                cocktail['name'] = item['name']
                cocktail["ingredients"] = item["recipeIngredient"]
                cocktail["instructions"] = []
                for instr in item["recipeInstructions"]:
                    if 'itemListElement' in instr.keys():
                        # Handle nested instructions
                        for element in instr['itemListElement']:
                            cocktail["instructions"].append(element["text"])        
                    else:
                        cocktail["instructions"].append(instr["text"])
                recipes.append(cocktail)

with open("./data/mybartender.json", 'w') as outfile:
    json.dump(recipes, outfile, indent=4)

In [ ]:
# The Webtender probably wants me to use a request with a user-agent
headers = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}
page = requests.get("https://www.webtender.com/db/browse?level=2&dir=drinks&char=%2A", headers=headers)


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">

<HTML>
<HEAD>
<BASE TARGET="_top">
</HEAD>

<BODY BGCOLOR="#bbbbbb" LINK="#330099" VLINK="#990033">
<B>
All drinks</B><HR>
<UL>
<LI> <A HREF="/db/drink/3622">The "Liquerice" Dream</A>
<LI> <A HREF="/db/drink/4511">'57 Chevy #2</A>
<LI> <A HREF="/db/drink/1736">'57 Chevy with a White License Plate</A>
<LI> <A HREF="/db/drink/4329">'57 Chevy</A>
<LI> <A HREF="/db/drink/2420">1-900-FUK-MEUP</A>
<LI> <A HREF="/db/drink/2434">110 in the shade</A>
<LI> <A HREF="/db/drink/2016">151 Florida Bushwacker</A>
<LI> <A HREF="/db/drink/2395">155 Belmont</A>
<LI> <A HREF="/db/drink/3794">187 URge</A>
<LI> <A HREF="/db/drink/4753">220 BTU</A>
<LI> <A HREF="/db/drink/3096">24k nightmare</A>
<LI> <A HREF="/db/drink/2366">252</A>
<LI> <A HREF="/db/drink/1671">3 Wise Men</A>
<LI> <A HREF="/db/drink/2372">3-Mile Long Island Iced Tea</A>
<LI> <A HREF="/db/drink/5356">3001</A>
<LI> <A HREF="/db/drink/4260">351 Special</A>
<LI> <A HREF="/db/drink/

In [ ]:
def get_webtender(url):
    time.sleep(random.random() * 2)
    headers = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}
    page = requests.get(url, headers=headers)

    drink_urls = []
    bs = BeautifulSoup(page.text)
    drinks = bs.find_all('a', href=re.compile("/db/drink/[0-9]+"))
    for drink in drinks:
        drink_url = f"https://www.webtender.com/db/{drink['href']}"
        drink_urls.append(drink_url)

    next = bs.find('a', text="Next")
    if next is not None:
        print(next['href'])
        url_next = f"https://www.webtender.com/db/{next['href']}"
        tmp_drinks = get_webtender(url_next)
        if tmp_drinks is not None:
            drink_urls.extend(tmp_drinks)

    return drink_urls



In [89]:
urls = get_webtender("https://www.webtender.com/db/browse?level=2&dir=drinks&char=%2A")

browse?level=2&dir=drinks&char=%2A&start=151
browse?level=2&dir=drinks&char=%2A&start=301
browse?level=2&dir=drinks&char=%2A&start=451
browse?level=2&dir=drinks&char=%2A&start=601
browse?level=2&dir=drinks&char=%2A&start=751
browse?level=2&dir=drinks&char=%2A&start=901
browse?level=2&dir=drinks&char=%2A&start=1051
browse?level=2&dir=drinks&char=%2A&start=1201
browse?level=2&dir=drinks&char=%2A&start=1351
browse?level=2&dir=drinks&char=%2A&start=1501
browse?level=2&dir=drinks&char=%2A&start=1651
browse?level=2&dir=drinks&char=%2A&start=1801
browse?level=2&dir=drinks&char=%2A&start=1951
browse?level=2&dir=drinks&char=%2A&start=2101
browse?level=2&dir=drinks&char=%2A&start=2251
browse?level=2&dir=drinks&char=%2A&start=2401
browse?level=2&dir=drinks&char=%2A&start=2551
browse?level=2&dir=drinks&char=%2A&start=2701
browse?level=2&dir=drinks&char=%2A&start=2851
browse?level=2&dir=drinks&char=%2A&start=3001
browse?level=2&dir=drinks&char=%2A&start=3151
browse?level=2&dir=drinks&char=%2A&start

In [92]:
with(open("webtender_urls.json", 'w')) as outfile:
    json.dump(urls, outfile)

In [101]:
# Accidentally malformed the URLS
for idx, url in enumerate(urls):
    urls[idx] = url.replace("/db/", "", 1)


In [125]:
url=random.choice(urls)
headers = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}
page = requests.get(url, headers=headers)
bs = BeautifulSoup(page.text)
if page.status_code == 200:
    print(page.text)
else:
    print(page.status_code)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">

<HTML>
<HEAD>
<TITLE>Oil Slick #2 (The Webtender)</TITLE>
<META http-equiv="PICS-Label" content='(PICS-1.1 "http://www.classify.org/safesurf/" l gen true for "http://www.webtender.com/" by "pics@webtender.com" r (SS~~000 6 SS~~006 1))'>
<META http-equiv="PICS-Label" content='(PICS-1.1 "http://www.rsac.org/ratingsv01.html" l gen true comment "RSACi North America Server" for "http://www.webtender.com/" on "1998.04.21T09:52-0800" r (n 0 s 0 v 0 l 1))'>
<LINK REL="top" HREF="/"><LINK REL="copyright" HREF="/info/c.html"><LINK REL="search" HREF="/search/">
<link rel="image_src" href="https://www.webtender.com/gfx/glasslogo.gif">
<meta property="fb:app_id" content="170290096347199">
<LINK REL="prev" HREF="/db/drink/4049"><LINK REL="next" HREF="/db/drink/4425"><meta property="og:title" content="Oil Slick #2">
<meta property="og:site_name" content="The Webtender">
<meta property="og:type" content="drink">
<meta property="og:url" co

In [127]:
recipes = []

for url in urls:
    time.sleep(random.random() * 2)
    headers = {"User-Agent":"Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:133.0) Gecko/20100101 Firefox/133.0"}
    page = requests.get(url, headers=headers)
    bs = BeautifulSoup(page.text)
    if page.status_code != 200:
        print(f"{page.status_code} {url}")
        continue

    cocktail = {}
    for titlecase in bs.find_all('h1'):
        if titlecase and titlecase.find('hr'):
            drink_name = titlecase.text
            cocktail['name'] = drink_name

    #Ingredients are in a list, so li tags, with a link to the ingredient name in it
    for l_item in bs.find_all('li'):
        ingredients = []
        if l_item and l_item.find('a'):
            ingredient = l_item.find('a').text.strip()
            # Ignore the other stuff that's in LI tags
            if ingredient in ["Find similar drinks", "Print drink", "Add to favourites", "Show your favourites"]:
                continue
        
            amount = l_item.text.replace(ingredient, '').strip()
            ingredients.append({"ingred_name": ingredient, "ingred_amount": amount})
        cocktail['ingredients'] = ingredients

    # Mixing instructions are in a paragraph after the H3 with the content "Mixing instructions:
    instr_head = bs.find('h3', text="Mixing instructions:")
    instr = instr_head.find_next('p')
    cocktail['instructions'] = [instr.text]

    # Save everything
    recipes.append(cocktail)


with(open("webtender.json", 'w')) as outfile:
    json.dump(recipes, outfile, indent=4)